In [24]:
import re
import pandas as pd
from sqlalchemy import create_engine, text
import time
import json

In [41]:
with open(r'../yavki_paroli.txt') as f:
    yavki = json.loads(f.read().replace("'",'"').replace('\n',''))  

In [42]:
yavki

{'user': 'nyzhur',
 'pwd': 'GfkPass121293',
 'host': '10.10.0.111',
 'port': 5432,
 'db': 'pet'}

In [43]:
db_config = yavki

connection_string = f'postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}'
engine = create_engine(connection_string)
conn = engine.connect()

In [5]:
def make_name(name):
    return "flth_" + re.sub('[^0-9a-zA-Z]+','_',name.lower())

In [6]:
def make_quere(query, conection):
    if conection._still_open_and_dbapi_connection_is_valid == True:
        conection.execute(text(query))
        conection.commit()
    else:
        print('Не получилось, пробую ещё раз')
        conection.rollback()
        conection.execute(text(query))
        conection.commit()
        
def get_anser(query):
   return pd.read_sql_query(query, con=engine)    

In [7]:
def check_condtion(condition):
    try:
        res = get_anser(f'''select count(distinct hhkey)
                               from cps.household_pet
                               where {condition}''').values[0][0]
        if res ==0:
            return  ValueError("Условие пустое")
        else:
            print('Всего', res)
            return res
        
    except:
        print('Ошибка условия')
        raise ValueError( 'Ошибка условия')

In [8]:
def get_id_flth(claster,filter_lable, conn):
    claster_list =['cmt_international_cluster_3','cmt_retail_cluster','ast','cmt_international_cluster_2','cmt_international_cluster_1','cmt_local_cluster', 'cmt_pharma_cluster']
    if claster not in claster_list:
      raise ValueError('Кластер указан неверно')
    filter_name = make_name(filter_lable)
    id_flt = get_anser(f""" select id_flt
FROM metadata.ctlg_flt
where table_name = '{filter_name}' and table_schema = '{claster}'
""")
    if len(id_flt) == 1:
     print('Уже есть! ID', id_flt.values[0][0])
     return id_flt.values[0][0]
    elif len(id_flt) == 0:
     make_meta = f"""INSERT INTO metadata.ctlg_flt (flt_type, table_schema, table_name, flt_label, flt_path )
    VALUES ('flt-h', '{claster}', {filter_name}, '{filter_lable}','/' );"""
     make_quere(make_meta, conn)
     print('Добавили мету')
     time.sleep(1)
     return get_id_flth(claster,filter_lable,conn) 

In [9]:
def make_flth(claster, filter_lable, condition, conn):
    filter_name = make_name(filter_lable)
    print(claster,filter_name)
    check = check_condtion(condition)
    id_flth = get_id_flth(claster, filter_lable, conn)
    make_filter = f"""CREATE OR REPLACE VIEW {claster}.{filter_name} AS
select {id_flth} as id_flt,
hp.hhkey,
dt_record AS dt_year
from cps.household_pet hp
where {condition}
"""
    make_quere(make_filter, conn)
    new_flth = get_anser(f'''select count(distinct hhkey)
from {claster}.{filter_name}''').values[0][0]
    if new_flth != check:
        return f"Ошибка {claster}.{filter_name}, условие не совпадает с созданным фильром "
    else:
        make_right=f"""GRANT UPDATE, TRIGGER, TRUNCATE, REFERENCES, INSERT, SELECT, DELETE ON TABLE {claster}.{filter_name} TO "msr.shinyproxy.svc";"""
        make_quere(make_right, conn)

        new_owener = f"""ALTER table {claster}.{filter_name}  OWNER TO gfk_developers;"""
        make_quere(new_owener, conn)
        
        
        return print(f"Фильтр {claster}.{filter_name} готов!")

In [10]:
def make_list(s:str, qu=False):
    if s.count('\n') < s.count(',')  :
        return [w.strip() for w in  s.replace('[','').replace(']','').replace('\n','').split(',')]
    else:
        if qu==False:
            return s.replace('[','').replace(']','').split('\n')
        else:    
            return [ f"'{w.strip()}'" for w in s.replace('[','').replace(']','').split('\n') ]


conn.rollback()

#Фильтры

In [11]:
#Community Size
com =  '"Community Size 2013"'
#com =  '"Community Size 2020"'


com_values = make_list("""10-49 ths
> 1 mln
250-499 ths
500-999 ths
50-249 ths
Moscow
Rural
St.Petersburg """)



com_cond_spec = f"""{com} in ({com_values})""".replace('[','').replace(']','')
com_urban = f"""{com} <> 'Rural' """
com_derevnya = f""" {com} = 'Rural' """

In [12]:
#Субъект
reg_list =make_list("""Moscow, St. Petersburg, Moskovskaya oblast, Leningradskaya oblast""") 
reg_cond = f""" "Federation Subject" in({reg_list})""".replace('[','').replace(']','') 


In [13]:
#Округ 
fo_list =make_list("""Moscow, Central, North-West, St. Petersburg, Privolzhie, South, Far East""") 
fo_cond = f""" "Federal Okrug" in({fo_list})""".replace('[','').replace(']','') 
fo_cond


' "Federal Okrug" in(\'Moscow\', \'Central\', \'North-West\', \'St. Petersburg\', \'Privolzhie\', \'South\', \'Far East\')'

In [14]:
baby_month = (12,18)
#baby_features = '"Age of Baby wo BR"'
baby_features = '"Age of Baby"'
months = ', '.join([f"'{str(i).zfill(2)} monthes'" for i in range(baby_month[0],baby_month[1]+1)])
baby_cond = f""" {baby_features} in ({months}) """

In [15]:
#mommy
mommy_list = make_list('''1st time
1st time + baby < 49 mth.
2nd time
2nd time + baby < 49 mth.
N/A
NA''')

momy_features = '"Maternity"'
momy_cond = momy_features + f' in ({mommy_list[2:4]})'.replace('[','').replace(']','')

In [16]:
#Age of 
ages = (0, 3)

age_feature =make_list("""Age of 2nd person
Age of 3rd person
Age of 4th person
Age of 5th person
Age of 6th person
Age of 7th person
Age of 8th person
Age of 9th person
Age of 10th person""") #Age of 1st person

cond = f"""NULLIF("{age_feature}", 'N/A'::text)::numeric >= {ages[0]}::numeric AND NULLIF("{age_feature}", 'N/A'::text)::numeric <= {ages[1]}::numeric"""
age_final_cond = []
for person in age_feature:
    cond = f"""NULLIF("{person}", 'N/A'::text)::numeric >= {ages[0]}::numeric AND NULLIF("{person}", 'N/A'::text)::numeric <= {ages[1]}::numeric OR"""
    age_final_cond.append(cond) 


age_final_cond = '\n'.join(age_final_cond)[:-3]
print(age_final_cond)

NULLIF("Age of 2nd person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 2nd person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 3rd person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 3rd person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 4th person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 4th person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 5th person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 5th person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 6th person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 6th person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 7th person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 7th person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 8th person", 'N/A'::text)::numeric >= 0::numeric AND NULLIF("Age of 8th person", 'N/A'::text)::numeric <= 3::numeric OR
NULLIF("Age of 9th person", 'N/A'::text)::numeric >= 0:

In [17]:
#Hosewife
housewife_values = [ ['up to 19 year','20-29 years'],
['30-39 years'],
['40-49 years'],
['50-65  years'],
['66 years and older']]

wife_lables = make_list("up to 29, 30-39, 40-49, 50-65, 65+")




In [18]:
cond = f" \"Age of housewife\"  in ({housewife_values[3]})".replace('[','').replace(']','')
check_condtion(cond)
print(cond)

Всего 8573
 "Age of housewife"  in ('50-65  years')


In [19]:
#money
money_fit = "HH net income 2018"

money_values =make_list("""up to 10000 rbls.
10001-14000 rbls.
14001-19000 rbls.
19001-27000 rbls.
27001-45000 rbls.
45001-60000 rbls.
60001-80000 rbls.
80001-100000 rbls.
more than 100000 rbls.""")

In [20]:
conn.rollback()

In [21]:
#list of cond
# com_urban com_derevnya  baby_cond momy_cond fo_cond reg_cond

final_cond = fo_cond
print(final_cond) 

 "Federal Okrug" in('Moscow', 'Central', 'North-West', 'St. Petersburg', 'Privolzhie', 'South', 'Far East')


In [22]:
clastres = ['cmt_international_cluster_3',
 'cmt_retail_cluster',
 'ast',
 'cmt_international_cluster_2',
 'cmt_international_cluster_1',
 'South_Pepsico_custom',
 'cmt_local_cluster',
 'cmt_pharma_cluster']

In [23]:
final_cond

' "Federal Okrug" in(\'Moscow\', \'Central\', \'North-West\', \'St. Petersburg\', \'Privolzhie\', \'South\', \'Far East\')'

In [24]:
make_flth('cmt_retail_cluster','gl_vinlab_total_geo',final_cond, conn )

cmt_retail_cluster flth_gl_vinlab_total_geo
Всего 24760
Добавили мету
Уже есть! ID 328
Фильтр cmt_retail_cluster.flth_gl_vinlab_total_geo готов!
